In [ ]:
import sys
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])

0

In [17]:
import requests
import time
import random
import json
import re # regular expressions (pattern matching)
import pandas as pd
import sqlite3

In [ ]:
# Headers
HEADER = {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "baggage": "sentry-environment=Production,sentry-release=businessunitprofile-consumersite%402.4845.0,sentry-public_key=7ac98d0742b24421b3d38448c4bf1184,sentry-trace_id=877b8e1f2533423cbdc92d5ea699cecd",
    "cookie": "TP.uuid=0706ee8f-76f5-4442-b3a2-b3e3700ea380; OptanonAlertBoxClosed=2025-09-08T15:22:09.409Z; ajs_anonymous_id=098d2a00-1d3a-4d88-b314-85b77407ce79; amplitude_idundefinedtrustpilot.com=eyJvcHRPdXQiOmZhbHNlLCJzZXNzaW9uSWQiOm51bGwsImxhc3RFdmVudFRpbWUiOm51bGwsImV2ZW50SWQiOjAsImlkZW50aWZ5SWQiOjAsInNlcXVlbmNlTnVtYmVyIjowfQ==; lc_referrer_url=https%3A%2F%2Fwww.google.com%2F; fc_referrer_url=https%3A%2F%2Fwww.google.com%2F; _hjHasCachedUserAttributes=true; __adroll_fpc=090700da94544d0729382867614e2fb7-1757344931547; _fbp=fb.1.1757344932562.32325800100083557; _hjSessionUser_391767=eyJpZCI6Ijg0YmMxMjY4LTUzNzEtNWI0Mi1iNjg2LWM5NDVmODZlNDQ1NiIsImNyZWF0ZWQiOjE3NTczNDQ5Mjk2NTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjSessionUser_398288=eyJpZCI6ImYxNGQxOGEwLWY5YjEtNTk0NS05YTAyLWQxY2FhNzdiNGVkOSIsImNyZWF0ZWQiOjE3NTc0MDk2ODcxMzgsImV4aXN0aW5nIjp0cnVlfQ==; _biz_uid=15963d115de44399b6007b6572bc93a7; _gid=GA1.2.201371872.1757409687; _biz_flagsA=%7B%22Version%22%3A1%2C%22ViewThrough%22%3A%221%22%2C%22XDomain%22%3A%221%22%7D; _biz_nA=3; _biz_pendingA=%5B%5D; _ga_3TEL80JZSG=GS2.1.s1757409687$o1$g1$t1757410655$j52$l0$h0; amplitude_id_cfe705a69359b8a4c0049d061ee5787btrustpilot.com=eyJkZXZpY2VJZCI6IjBmMzNhMWNmLTc2OWEtNDIwZi04ZDU5LWYwOWYxYjhiZDFiNFIiLCJ1c2VySWQiOm51bGwsIm9wdE91dCI6ZmFsc2UsInNlc3Npb25JZCI6MTc1NzQxMDA2NzMyOSwibGFzdEV2ZW50VGltZSI6MTc1NzQxMTYyODc1OSwiZXZlbnRJZCI6MiwiaWRlbnRpZnlJZCI6MSwic2VxdWVuY2VOdW1iZXIiOjN9; _csrf=GSISE9yn_UcknXuQ4tWofQV_; _ga=GA1.1.909402240.1757344931; _gcl_au=1.1.1745043198.1757344930.1881067649.1757412719.1757412718; tp-external-referrer=%7B%22referrer%22%3A%22https%3A%2F%2Fwww.google.com%2F%22%2C%22entry%22%3A%22https%3A%2F%2Fie.trustpilot.com%2F%22%7D; OptanonConsent=isGpcEnabled=0&datestamp=Tue+Sep+09+2025+14%3A55%3A17+GMT%2B0100+(Irish+Standard+Time)&version=202503.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=27ed8391-26b9-4508-a0c3-079183ae230c&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&intType=1&geolocation=IE%3BL&AwaitingReconsent=false; __ar_v4=WB2AUOFO2ZFVFHJ5MEHUWO%3A20250908%3A16%7CIUNG4UKUTNBEXHEW2IT6BS%3A20250908%3A16%7C6U7APKK4E5F45KVW4MS4AL%3A20250908%3A16; _hjSession_391767=eyJpZCI6IjExNjc1NDg0LTJlODMtNGMzZS04MWIyLTliZTZmYTRlZWU5NSIsImMiOjE3NTc0MjcxNTg3MzMsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; amplitude_id_67f7b7e6c8cb1b558b0c5bda2f747b07trustpilot.com=eyJkZXZpY2VJZCI6IjA5OGQyYTAwLTFkM2EtNGQ4OC1iMzE0LTg1Yjc3NDA3Y2U3OSIsInVzZXJJZCI6bnVsbCwib3B0T3V0IjpmYWxzZSwic2Vzc2lvbklkIjoxNzU3NDI4NjM0OTY3LCJsYXN0RXZlbnRUaW1lIjoxNzU3NDI5MzU2Njk3LCJldmVudElkIjo1NSwiaWRlbnRpZnlJZCI6OCwic2VxdWVuY2VOdW1iZXIiOjYzfQ==; _ga_11HBWMC274=GS2.1.s1757425910$o8$g1$t1757429356$j31$l0$h0",
    "newrelic": "eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjQzODU4ODciLCJhcCI6IjUzODY4MjQ5OSIsImlkIjoiMmQxZmI0ODJjZTdlOWI4YSIsInRyIjoiNzUwMWEzZjVkZDliZWZhY2MyOTA3ODA4NWFkNThmNjEiLCJ0aSI6MTc1NzQyOTM4MTE5MX19",
    "priority": "u=1, i",
    "referer": "https://ie.trustpilot.com/review/www.asos.com?page=2",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sentry-trace": "877b8e1f2533423cbdc92d5ea699cecd-aa8c8d2f9d573f99",
    "traceparent": "00-7501a3f5dd9befacc29078085ad58f61-2d1fb482ce7e9b8a-01",
    "tracestate": "4385887@nr=0-1-4385887-538682499-2d1fb482ce7e9b8a----1757429381191",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.5 Safari/605.1.15",
    "x-nextjs-data": "1"
}


all_reviews = []

# Target: ASOS reviews, pages 2 to 15
target = 'www.asos.com'

for i in range(2, 15):  # pages 2 to 15
    url = f'https://ie.trustpilot.com/_next/data/businessunitprofile-consumersite-2.4845.0/review/www.asos.com.json?page={i}&businessUnit={target}'
    
    resp = requests.get(url, headers=HEADER)
    if resp.status_code != 200:
        print(f"Failed to fetch page {i}")
        continue
    
    res = resp.json()
    
    # Extract reviews from JSON
    reviews = res.get('pageProps', {}).get('reviews', [])
    all_reviews.extend(reviews)
    
    print(f'Page {i} fetched, {len(reviews)} reviews.')
    
    time.sleep(1)  # polite delay to avoid being blocked

# Normalize into DataFrame
df = pd.json_normalize(all_reviews)
df.to_csv('asos_reviews.csv', index=False)

print(f"Saved {len(all_reviews)} reviews to CSV")


Page 2 fetched, 20 reviews.
Page 3 fetched, 20 reviews.
Page 4 fetched, 20 reviews.
Page 5 fetched, 20 reviews.
Page 6 fetched, 20 reviews.
Page 7 fetched, 20 reviews.
Page 8 fetched, 20 reviews.
Page 9 fetched, 20 reviews.
Page 10 fetched, 20 reviews.
Page 11 fetched, 20 reviews.
Page 12 fetched, 20 reviews.
Page 13 fetched, 20 reviews.
Page 14 fetched, 20 reviews.
Saved 260 reviews to CSV


In [4]:
# load CSV and print head
df = pd.read_csv('asos_reviews.csv')
print(df.head())

                         id  filtered  pending  \
0  68b951e16796366d612c0a5a     False    False   
1  68b9508a183d3a0e366f4055     False    False   
2  68b93c80ca0e2a5da9e471f2     False    False   
3  68b8afadbca99a540889a0eb     False    False   
4  68b88f6f5a626c5041eea2c4     False    False   

                                                text  rating  \
0  I ordered tshirt of 83£ and did a partial retu...       1   
1  AWFUL! Still waiting for a refund that was app...       1   
2  Love asos fast delivery and selection - but th...       4   
3  Dreadful customer care. My order hasn’t arrive...       1   
4  I ordered a baseball cap from ASOS, as I am a ...       1   

                                      title  likes   source  report  \
0  Ordered tshirt of 83£ and did a partial…      0  Organic     NaN   
1                                     AWFUL      0  Organic     NaN   
2                          Great selection       0  Organic     NaN   
3                    Dreadful 

In [5]:
# print number of rows and collumns
print(df.shape)

(260, 34)


In [6]:
# print collumns name
print(df.columns.tolist())

['id', 'filtered', 'pending', 'text', 'rating', 'title', 'likes', 'source', 'report', 'hasUnhandledReports', 'reply', 'consumersReviewCountOnSameDomain', 'consumersReviewCountOnSameLocation', 'productReviews', 'language', 'location', 'labels.merged', 'labels.verification.isVerified', 'labels.verification.createdDateTime', 'labels.verification.reviewSourceName', 'labels.verification.verificationSource', 'labels.verification.verificationLevel', 'labels.verification.hasDachExclusion', 'dates.experiencedDate', 'dates.publishedDate', 'dates.updatedDate', 'dates.submittedDate', 'consumer.id', 'consumer.displayName', 'consumer.imageUrl', 'consumer.numberOfReviews', 'consumer.countryCode', 'consumer.hasImage', 'consumer.isVerified']


In [7]:
# check what is in column consumer image
print(df['consumer.imageUrl'].head(5))

0    https://user-images.trustpilot.com/686e4d994e6...
1                                                  NaN
2    https://user-images.trustpilot.com/68b93b6acc7...
3                                                  NaN
4    https://user-images.trustpilot.com/5d092243952...
Name: consumer.imageUrl, dtype: object


In [8]:

# print data type and Null values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      260 non-null    object 
 1   filtered                                260 non-null    bool   
 2   pending                                 260 non-null    bool   
 3   text                                    260 non-null    object 
 4   rating                                  260 non-null    int64  
 5   title                                   260 non-null    object 
 6   likes                                   260 non-null    int64  
 7   source                                  260 non-null    object 
 8   report                                  0 non-null      float64
 9   hasUnhandledReports                     260 non-null    bool   
 10  reply                                   0 non-null      float6

In [ ]:
# copy df
df_clean = df.copy()


In [10]:
# List of columns to drop
cols_to_drop = [
    'filtered', 'pending', 'source', 'report', 'hasUnhandledReports', 'reply',
    'consumersReviewCountOnSameDomain', 'consumersReviewCountOnSameLocation',
    'productReviews', 'language', 'location', 'labels.merged',
    'labels.verification.isVerified', 'labels.verification.createdDateTime',
    'labels.verification.reviewSourceName', 'labels.verification.verificationSource',
    'labels.verification.verificationLevel', 'labels.verification.hasDachExclusion',
    'dates.experiencedDate', 'dates.updatedDate', 'dates.submittedDate',
    'consumer.isVerified', 'consumer.id', 'consumer.imageUrl','consumer.hasImage'
    
]

# Drop columns
df_clean = df_clean.drop(columns=cols_to_drop, errors="ignore")

# Check remaining columns
print(df_clean.columns)

Index(['id', 'text', 'rating', 'title', 'likes', 'dates.publishedDate',
       'consumer.displayName', 'consumer.numberOfReviews',
       'consumer.countryCode'],
      dtype='object')


In [ ]:
# checking change
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        260 non-null    object
 1   text                      260 non-null    object
 2   rating                    260 non-null    int64 
 3   title                     260 non-null    object
 4   likes                     260 non-null    int64 
 5   dates.publishedDate       260 non-null    object
 6   consumer.displayName      260 non-null    object
 7   consumer.numberOfReviews  260 non-null    int64 
 8   consumer.countryCode      260 non-null    object
dtypes: int64(3), object(6)
memory usage: 18.4+ KB


In [14]:
# change columns name
df_clean = df_clean.rename(columns={
    "dates.publishedDate": "date_published",
    "consumer.numberOfReviews": "number_of_review",
    "consumer.countryCode": "country_code",
    "consumer.displayName":"name"
})

# change date_published (object) columns to datetime
df_clean["date_published"] = pd.to_datetime(df_clean["date_published"], errors="coerce")


In [ ]:
# checking change
# ns → nanosecond precision 
# UTC → the datetimes are timezone-aware, stored in Coordinated Universal Time.
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                260 non-null    object             
 1   text              260 non-null    object             
 2   rating            260 non-null    int64              
 3   title             260 non-null    object             
 4   likes             260 non-null    int64              
 5   date_published    260 non-null    datetime64[ns, UTC]
 6   name              260 non-null    object             
 7   number_of_review  260 non-null    int64              
 8   country_code      260 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 18.4+ KB


In [ ]:

# clean text column
def clean_text(text):
    if pd.isna(text):
        return ""
    # lowercase
    text = text.lower()
    # remove URLs
    # re (regular expressions) library
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    return text

# Apply cleaning and create new column
df_clean["text_clean"] = df_clean["text"].apply(clean_text)

# Check result
print(df_clean[["text", "text_clean"]].head())


                                                text  \
0  I ordered tshirt of 83£ and did a partial retu...   
1  AWFUL! Still waiting for a refund that was app...   
2  Love asos fast delivery and selection - but th...   
3  Dreadful customer care. My order hasn’t arrive...   
4  I ordered a baseball cap from ASOS, as I am a ...   

                                          text_clean  
0  i ordered tshirt of 83£ and did a partial retu...  
1  awful! still waiting for a refund that was app...  
2  love asos fast delivery and selection - but th...  
3  dreadful customer care. my order hasn’t arrive...  
4  i ordered a baseball cap from asos, as i am a ...  
